# Change in late season snow cover in the Rocky mountains

### Analysis of June snowpack in the high rocky mountains, comparing the average of 2001-2005 to 2021-2025



In [2]:
import ee
import folium
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image, display
import json
import sys
sys.path.append('../Setup')  # Go up one level to reach Setup folder
from gee_setup import *

setup_gee()

from grand_file import snow_difference_map

✅ earthengine-api imported successfully
✅ folium imported successfully
✅ pandas imported successfully
✅ numpy imported successfully
✅ matplotlib imported successfully
✅ seaborn imported successfully
✅ IPython display imported successfully
✅ json imported successfully (built-in)
GEE utilities loaded successfully!
Use quick_setup() for one-line initialization
✅ Matplotlib configured
Google Earth Engine initialized successfully!
EE version: 1.6.3
✅ Matplotlib configured
Google Earth Engine initialized successfully!
EE version: 1.6.3
✅ Matplotlib configured
Google Earth Engine initialized successfully!
EE version: 1.6.3
✅ Matplotlib configured
Google Earth Engine initialized successfully!
EE version: 1.6.3
✅ Matplotlib configured
Google Earth Engine initialized successfully!
EE version: 1.6.3
Google Earth Engine initialized successfully!
EE version: 1.6.3


In [4]:
## define aoi polygon
aoi = ee.Geometry.Polygon([[
    [-106.99065650552905, 39.088354116315266],
    [-106.71050513834155, 38.652122067998306], 
    [-106.14470923990405, 38.596331654032184],
    [-106.40838111490405, 39.21189167999414],
    [-106.99065650552905, 39.088354116315266]
]])
snow_difference_map(aoi)

Year: 1990, Image Count: 2
Year: 1991, Image Count: 3
Year: 1992, Image Count: 1
Year: 1993, Image Count: 3
Year: 1992, Image Count: 1
Year: 1993, Image Count: 3
Year: 1994, Image Count: 3
Year: 1995, Image Count: 2
Year: 1994, Image Count: 3
Year: 1995, Image Count: 2
Year: 1996, Image Count: 1
Year: 1997, Image Count: 1
Year: 1996, Image Count: 1
Year: 1997, Image Count: 1
Year: 1998, Image Count: 1
Year: 1999, Image Count: 2
Year: 1998, Image Count: 1
Year: 1999, Image Count: 2
Year: 2015, Image Count: 3
Year: 2015, Image Count: 3
Year: 2016, Image Count: 1
Year: 2016, Image Count: 1
Year: 2017, Image Count: 2
Year: 2018, Image Count: 2
Year: 2017, Image Count: 2
Year: 2018, Image Count: 2
Year: 2022, Image Count: 1
Year: 2023, Image Count: 1
Year: 2022, Image Count: 1
Year: 2023, Image Count: 1
Year: 2024, Image Count: 1
Year: 2024, Image Count: 1
✅ Interactive map saved as 'snow_difference_analysis.html'
✅ Interactive map saved as 'snow_difference_analysis.html'
